## http://hgdownload.cse.ucsc.edu/goldenPath/hg38/vsPanTro5/
## hg38.panTro5.net.axt.gz    04-Aug-2016 01:50  1.5G 

** Second Version
** counts no-changes, and when there is more than 1 difference between 2 n-mers
** also, for each n-mer, normalize by the total n-mer count in the genome

** original Test, does not count no-changes, or when there is more than 1 difference between 2 n-mers

In [17]:
import csv
from itertools import product

import datetime as dt
n1=dt.datetime.now()

bases=['A','C','G','T']
k = 6
kmer_dict = dict()
kmer_list = [''.join(p) for p in product(bases, repeat=k)]
for i in kmer_list:
    kmer_dict[i] = dict()
    for j in kmer_list:
        kmer_dict[i][j] = 0

file = open('/home/alisay/hexamer_mutation_data/hg38.panTro5.net.axt', "r+")
out_file = open('/home/alisay/hexamer_mutation_data/hg38.panTro5.six_mers', "w")
thedatawriter = csv.writer(out_file)

while True:
    line = file.readline()
    if not line: break
    if line.startswith("#") or line.startswith("\n"):
        continue
    human = file.readline().strip().upper()
    if not human: break
    chimp = file.readline().strip().upper()
    for idx in range(0,len(human)):
        if ((human[idx] != chimp[idx]) and (human[idx] != "-") and (chimp[idx] != "-")):
            #print(idx," ",human[idx]," ",chimp[idx],"\n")
            for start in range(max(0,idx-k+1),min(idx+1,len(human)-k+1)):
                h_kmer = human[start:(start+k)]
                c_kmer = chimp[start:(start+k)]
                if (("-" not in h_kmer) and ("-" not in c_kmer) and ("N" not in h_kmer) and ("N" not in c_kmer)
                   and (sum(ch1 != ch2 for ch1, ch2 in zip(h_kmer, c_kmer)) == 1)):
                    #if (h_kmer == "AA" and c_kmer == "TT"):
                    #print(line)
                    #print(idx)
                    #print(start)
                    kmer_dict[h_kmer][c_kmer] += 1 

#output
thedatawriter.writerow(["kmer"] + kmer_list)
for i in kmer_list:
    row = [i]
    for j in kmer_list:
        row.append(kmer_dict[i][j])
        #print(i, " ", j, " ", kmer_dict[i][j])
    thedatawriter.writerow(row)
out_file.close()  

n2=dt.datetime.now()
print((n2-n1).seconds)

1747
